In [1]:
import os, sys

os.chdir("../../..")

In [2]:
print(os.path.abspath("./"))

/data2/huping/home/projects/src/github.com/any35/MOS


In [5]:
# make mae dataset, (只保留图像信息, 不要mask)
import torch, glob, os

os.makedirs(".cache/dataset/mae-dataset/", exist_ok=True)

valid_set = set(["VarDA.pt"])

ignore_set = set(["CMRI.pt"])

train_all_images = []
valid_all_images = []
cmri_train_images_index = [[] for _ in range(10)]
cmri_valid_images_index = [[] for _ in range(10)]

src_dir = ".cache/dataset/mmae-dataset/split-1.5mm"

file_list = glob.glob(f"{src_dir}/*.pt")
file_list.sort()

for file in file_list:
    file_name = file.split("/")[-1]
    if file_name in ignore_set:
        continue

    is_valid_set = file_name in valid_set

    data = torch.load(file)
    image, meta = data["image"], data["image_meta"]
    assert len(image.shape) == 3, f"{image.shape}, {file}"  # bs, h ,w
    index_list = []
    for i, image_type in enumerate(meta[:, 1].tolist()):
        if image_type < 10:  # 0~9为各类图像
            index_list.append(i)
    image = image[index_list]

    if is_valid_set:
        print("valid images:", file)
        valid_all_images.append(image)
    else:
        train_all_images.append(image)

    print(f"processed {file}...\t {image.shape[0]} images")

# cmri images
data: dict[str, torch.Tensor] = torch.load(f"{src_dir}/CMRI.pt")
# (bs, h, w), (bs, [uid, type])
cmri_image_list, meta_list = data["image"], data["image_meta"]

index_list = []
for i, meta in enumerate(meta_list.tolist()):
    uid, image_type = meta
    if image_type >= 10 or image_type < 0:  # 0~9为各类图像
        continue

    partition = uid % 10

    for s in range(10):
        if s == partition:
            cmri_valid_images_index[s].append(len(index_list))
        else:
            cmri_train_images_index[s].append(len(index_list))

    index_list.append(i)

cmri_image_list = cmri_image_list[index_list]

train_all_images = torch.cat(train_all_images, dim=0).to(torch.uint8)
valid_all_images = torch.cat(valid_all_images, dim=0).to(torch.uint8)

cmri_train_images_index = [torch.tensor(t) for t in cmri_train_images_index]
cmri_valid_images_index = [torch.tensor(t) for t in cmri_valid_images_index]

data = {
    "image": train_all_images,  # 训练集的图像
    "cmri_image": cmri_image_list,  # cmri的全部图像(微调的时候用)
}
for i in range(10):
    data[f"cmri_image_index/{i}"] = cmri_train_images_index[i]  # cmri微调的时候选择的图像id, for 10 folds valid
torch.save(data, ".cache/dataset/mae-dataset/dataset-train.pt")

data = {
    "image": valid_all_images,
    "cmri_image": cmri_image_list,
}
for i in range(10):
    data[f"cmri_image_index/{i}"] = cmri_valid_images_index[i]  # cmri微调的时候选择的验证图像, for 10 folds valid

torch.save(data, ".cache/dataset/mae-dataset/dataset-valid.pt")


print(f"total {train_all_images.shape[0]} train images, {valid_all_images.shape[0]} valid images!")
del train_all_images, valid_all_images

processed .cache/dataset/mmae-dataset/split-1.5mm/ACDC.pt...	 38346 images
processed .cache/dataset/mmae-dataset/split-1.5mm/CMRI-private.pt...	 48195 images
processed .cache/dataset/mmae-dataset/split-1.5mm/CT_MR_2D_Dataset_DA.pt...	 1568 images
processed .cache/dataset/mmae-dataset/split-1.5mm/CoA_MRIData.pt...	 2641 images
processed .cache/dataset/mmae-dataset/split-1.5mm/DOI_10.7910_DVN_CI3WB6.pt...	 42225 images
processed .cache/dataset/mmae-dataset/split-1.5mm/DOI_10.7910_DVN_JMZHVI.pt...	 16000 images
processed .cache/dataset/mmae-dataset/split-1.5mm/DOI_10.7910_DVN_N1R1Q4.pt...	 1050 images
processed .cache/dataset/mmae-dataset/split-1.5mm/EMIDEC.pt...	 1066 images
processed .cache/dataset/mmae-dataset/split-1.5mm/HVSMR2016.pt...	 3600 images
processed .cache/dataset/mmae-dataset/split-1.5mm/LeftAtrialSegmentationChallenge2013.pt...	 3568 images
processed .cache/dataset/mmae-dataset/split-1.5mm/LeftAtrialSegmentationKaggle.pt...	 2271 images
processed .cache/dataset/mmae-datase